In [1]:
import json
import pandas as pd

# Load the JSON file
with open("dev_tables.json", "r", encoding="utf-8") as f:
    data = json.load(f)

table_schemas = {}
dbs = []

for db in data:
    db_id = db.get("db_id", [])
    dbs.append(db_id)
    schema_promt = f"Below are the descriptions for the database {db_id}:\n"
    for table in db.get("table_names_original", []):
        # import csv file in dev_databases/{db_id}/database_description/{table}.csv
        try:
            table_desc = pd.read_csv(f"dev_databases/{db_id}/database_description/{table}.csv")
        except Exception as e:
            print(db_id, table, e)
        table_index = db["table_names_original"].index(table)
        schema_promt += f"[Table name]\n{table}\n[(Column name, description, type)]\n"
        for i, (tbl_idx, col_name) in enumerate(db.get("column_names_original", [])):
            if tbl_idx == table_index:
                # find the column description whose original_column_name is col_name
                try:
                    col_desc = table_desc[table_desc["original_column_name"] == col_name]["column_description"].values[0]
                except Exception as e:
                    col_desc = ""
                type_desc = db.get("column_types", [])[i]
                schema_promt += f"{col_name}, {col_desc}, {type_desc}\n"
        schema_promt += "\n"
    table_schemas[db_id] = schema_promt

In [71]:
import json

def next():
    with open(f"eval_results_schema_desc.json", "r", encoding="utf-8") as f:
        queries = json.load(f)
    for query in queries["student_club"]:
        yield query
generator = next()
results = []

In [130]:
query = generator.__next__()
print(query["question"])

Indicate the top source of funds received in September 2019 based on their amount.


In [ ]:
from openai import OpenAI

client = OpenAI(
    
)

f_SQL = "SELECT notes WHERE income = 'fundraising' time = 2019/9/14"
#generate full prompt for queries
prompt = table_schemas["student_club"]
prompt += f"Given the above database schema, convert the following pseudo-SQL query to a executable SQL:\n"
prompt += f"{f_SQL}\nReturn a single SQL with no explanation.\nBegin!"
response = ""
try:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        temperature=0
    )
    response = completion.choices[0].message.content
except Exception as e:
    pass
print(response)
# query["f_SQL"] = f_SQL
# query["new_generated_query"] = response
# results.append(query)

```sql
SELECT notes FROM income WHERE source = 'fundraising' AND date_received = '2019-09-14';
```


In [132]:
with open("f_SQL_student_club.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)